# Implementation of an bully algorithm without the Django framework
### This is a test, to know how it's build without the extra layers -> object based Python classes

In [7]:
from random import randint
from threading import Thread

In [8]:
class TruckThreading(Thread):
    class __Truck__(object):
        def __init__(self, truckId: int, convoyPosition: int, convoyLeaderId: int, speed: float, broke: bool):
            self.truckId = truckId
            self.convoyPosition = convoyPosition
            self.convoyLeaderId = convoyLeaderId
            self.speed = speed
            self.broke = broke

        """
        def startPoll(self):
            #answer = {'pos': ('alive', 'truckId')}
            oldConvoyPosition = self.convoyPosition
            
            
            # BIG SHIT HERE
            trucks = self.convoyReferenceList.getTrucks()
            trucksAlive = {}
            for convoyPosition, truck in trucks.items():
                if convoyPosition <= self.convoyPosition:
                    trucksAlive[convoyPosition] = (truck.alive(), truck.getTruckId())
            newConvoyPosition = self.__bully__(trucksAlive)
            self.convoyPosition = newConvoyPosition
            return oldConvoyPosition - newConvoyPosition

        def __bully__(self, trucksAlive):
            newPositions = []
            for position, (alive, truckId) in trucksAlive.items():
                if alive:
                    newPositions.append(truckId)
            return newPositions.index(self.truckId)
            #self.truckId = newPositions.index(self.truckId)
        """

    def __init__(self, truckId: int, convoyPosition: int, convoyLeaderId: int, speed: float, broke: bool):
        self.__alive__ = True
        self.__truck__ = self.__Truck__(truckId, convoyPosition, convoyLeaderId, speed, broke)
    
    def run(self):
        while self.__alive__:
            pass
    
    def stop(self):
        self.__alive__ = False
    
    def alive(self):
        return self.__truck__.broke

    def toggleBroke(self):
        self.__truck__.broke = not self.__truck__.broke
        
    def getTruckId(self):
        return self.__truck__.truckId
    
    def setConvoyPosition(self, convoyPosition):
        self.__truck__.convoyPosition = convoyPosition
        
    def getConvoyPosition(self):
        return self.__truck__.convoyPosition
    
    def setConvoyLeaderId(self, convoyLeaderId):
        self.__truck__.convoyLeaderId = convoyLeaderId
        
    def getConvoyLeaderId(self):
        return self.__truck__.convoyLeaderId
    
    def setSpeed(self, speed):
        self.__truck__.speed = speed
        
    def getSpeed(self):
        return self.__truck__.speed


In [9]:
class ConvoyListThreading(Thread):
    class __ConvoyReferenceList__(object):
        def __init__(self):
            self.convoyList = {}
    
    def __init__(self):
        self.__alive__ = True
        self.__convoyReferenceList__ = self.__ConvoyReferenceList__()
    
    def run(self):
        while self.__alive__:
            pass
    
    def stop(self):
        self.__alive__ = False
    
    def getAllTruckAdresses(self):
        """@return list of adresses of the trucks in this convoy"""
        return self.__convoyReferenceList__.convoyList.keys()
    
    def getAllTruckIds(self):
        """@return list of ids of the trucks in this convoy"""
        return self.__convoyReferenceList__.convoyList.values()
    
    def getTruckAdress(self, truckId: int):
        """@return resolved networkadress for the requested truckId"""
        return self.__convoyReferenceList__.convoyList.get(truckId)
    
    def addTuck(self, truckId: int, adress: str):
        """@return success"""
        if not truckId in self.__convoyReferenceList__.convoyList.values():
            self.__convoyReferenceList__.convoyList[truckId] = adress
            return True
        return False
        
    def removeTruck(self, truckId: int):
        """@return success"""
        if truckId in self.__convoyReferenceList__.convoyList.values():
            del self.__convoyReferenceList__.convoyList[truckId]
            return True
        return False


In [10]:
def initTrucks(count: int, initialSpeed: float, initialBroke: bool):
    leaderId = 0
    truckIds = []
    trucks = []
    
    for counter in range(count):
        newConvoyPosition = counter
        newTruckId = randomTruckId(truckIds)
        truckIds.append(newTruckId)
        
        if counter == 0:
            leaderId = newTruckId
        
        trucks.append(
            TruckThreading(
                truckId = newTruckId,
                convoyPosition = newConvoyPosition,
                convoyLeaderId = leaderId,
                speed = initialSpeed,
                broke = initialBroke
            )
        )
    return (trucks, truckIds)
    
def initConvoyList(trucks: list):
    convoyList = ConvoyListThreading()
    inc = 0
    for truck in trucks:
        adress = str(inc)
        success = convoyList.addTuck(truck.getTruckId(), adress)
        if not success:
            raise ValueError(f'Can\'t reach truck with id \'{truck.getTruckId()}\' and adress \'{adress}\'')
        inc += 1
    return convoyList

def randomTruckId(truckIds: list):
    success = False
    while not success:
        newId = randint(1, 1000)
        if not newId in truckIds:
            success = not success
    return newId


In [11]:
__name__ = '__main__'

AMOUNT_OF_TRUCKS = 5
INITIAL_SPEED = 30.0 # km/h
INITIAL_BROKE = False

TRUCK_IDS = []
TRUCKS = []
CONVOY_LIST = False

if __name__ == '__main__':
    TRUCKS, TRUCK_IDS = initTrucks(AMOUNT_OF_TRUCKS, INITIAL_SPEED, INITIAL_BROKE)
    
    if TRUCKS and TRUCK_IDS:
        print(TRUCK_IDS)
    
    CONVOY_LIST = initConvoyList(TRUCKS)
    
    if CONVOY_LIST:
        print(CONVOY_LIST.__convoyReferenceList__.convoyList)
        print(CONVOY_LIST.getAllTruckAdresses())
        print(CONVOY_LIST.getAllTruckIds())

# https://www.youtube.com/watch?v=Tf1DEI2lEe0


[763, 279, 750, 965, 61]
{763: '0', 279: '1', 750: '2', 965: '3', 61: '4'}
dict_keys([763, 279, 750, 965, 61])
dict_values(['0', '1', '2', '3', '4'])
